In [ ]:
import Levenshtein as LV
import time
import csv
import os
import tkinter as tk
from tkinter import *
from tkinter import filedialog,ttk
import shutil
#========================= define functions ==============================
def m_choice(qc_switch,lc_mth):
    options = [('Skip Quality Control','Start from a Fixed Position'),('Skip Quality Control','Find with Conserved Seq before'),('Skip Quality Control','Find with Conserved Seq after'),('Perform Quality Control','Start from a Fixed Position'),('Perform Quality Control','Find with Conserved Seq before'),('Perform Quality Control','Find with Conserved Seq after')]
    return options.index((qc_switch,lc_mth))
def qualityfilter(threshold,tquant,qual):
    output = True
    count = 0
    for i in range(len(qual)):
        basequal = ord(qual[i])
        if basequal < threshold:
            count += 1
        if count > tquant:
            output = False
            break
    return output
def temp_output(datalist,barlist, mismatcheddata,lowqualdata,i,new_folder,start):
    csvfile = open(new_folder+'mismatched data.txt','a',newline='')
    write = csv.writer(csvfile)
    write.writerows(mismatcheddata)
    if lowqualdata != []:
        csvfile = open(path.replace('.fastq','/')+'low quality data.txt','a',newline='')
        write = csv.writer(csvfile)
        write.writerows(lowqualdata)
    for idx in range(len(barlist)):
        csvfile = open(new_folder+'Barcode_'+barlist[idx]+'.txt','a',newline='')
        write = csv.writer(csvfile)
        write.writerows(datalist[idx])
    if i == 200001:
        testtime = time.process_time()
        print('>> Estimated time for processing 1,000,000 sequences requires '+str(int(20*(testtime-start))+1)+' s.')
    print('>> Sorted '+str(int(i/4000))+'000 sequences.')
    return True
def BClocate(BC_seq,template_seq): #输入序列var_seq,查找在模板序列template_seq中的位置,返回实际位数
    try:
        out = template_seq.index(BC_seq)
    except:
        out = -1
        valmin = 1
        for k in range(len(template_seq)-len(BC_seq)):
            val = LV.distance(BC_seq,template_seq[k:k+len(BC_seq)])
            valmin = min(val,valmin)
            if valmin == val:
                out = k
    return out
#========================= define functions ==============================

#========================= define GUI functions ==============================
def preview_fastq():
    file_path = filedialog.askopenfilename(filetypes=[("fastq files", "*.fastq"),('fasta file','*.fasta'),('fq file','*.fq')])
    first_five_lines = ''
    count,count2 = 0,1
    with open(file_path, 'r',buffering = 1) as f1:
        for line in f1:
            if count%4 == 1:
                first_five_lines += str(count2)+ '. '+line
                count2 += 1
            count += 1
            if count == 41:
                break
    file_path_entry.delete(0, tk.END)
    file_path_entry.insert(0, file_path)
    text_box.delete('1.0', tk.END)
    text_box.insert(tk.END, first_five_lines)
def submit():
    file_path = file_path_entry.get()  # input file path
    barlist = barcode1_entry.get().upper().strip(' ,()\n').split(',') # barcode list
    qc_switch = qc1.get()  # quality check switch, on or off
    qc_minq = qc_entry.get() # minimal quality
    qc_maxb = qc2_entry.get() # maximal low quality bases
    lc_mth = dropdown.get() # barcode locating method
    pos = fp_entry.get() # barcode position (fixed)
    try:
        pos = int(pos)-1
    except:
        True
    cons_bef = cb_entry.get().upper()  # conserved seq before barcode 
    cons_aft = ca_entry.get().upper()  # conserved seq after barcode 
    mode = m_choice(qc_switch,lc_mth)
    folder_path = file_path.replace(file_path.split(os.sep)[-1],'')
    new_folder = file_path[:file_path.index('.f')]
    if os.path.exists(new_folder) == True:
        shutil.rmtree(new_folder)
    os.mkdir(new_folder)
    new_folder += os.sep
    for i in barlist:
        csvfile = open(new_folder+'Barcode_'+i+'.txt','w',newline='')
        write = csv.writer(csvfile)
    csvfile.close()
    print('>> Created .txt files for storage of sequences with different barcodes.')
    datalist = []
    for i in range(len(barlist)):
        datalist.append([])
    i = 0
    datalist_empty = datalist
    mismatcheddata = []
    if mode < 3:
        print('>> Not performing quality check.')
    else:
        print('>> Perform quality check, minimal quality threshold = '+ qc_minq +', maximal low quality bases = '+ qc_maxb+'.')
        qc_minq = int(qc_minq)
        qc_maxb = int(qc_maxb)
    lowqualdata = []
    start = time.process_time()
    ########################################
    with open(file_path,'r',buffering = 1) as f:
        if mode == 0:    # no quality control, position search
            for line in f:
                i += 1
                if i%4 == 2:
                    seq = line.upper().strip(' \n')
                    barseq = seq[pos:pos+len(barlist[0])]
                    try:
                        idx = barlist.index(barseq)
                        datalist[idx].append([seq])
                    except:
                        mismatcheddata.append([seq,'barcode not found'])
                if i%200000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismatcheddata,lowqualdata,i,new_folder,start)
                    mismatcheddata,lowqualdata = [],[]
                    datalist = datalist_empty
        if mode == 1:   # no quality control, conserved seq before barcode
            for line in f:
                i += 1
                if i%4 == 2:
                    seq = line.upper().strip(' \n')
                    try:
                        pos = seq.index(cons_bef)+len(cons_bef)
                    except:
                        mismatcheddata.append([seq,'conserved seq not found'])
                        continue
                    barseq = seq[pos:pos+len(barlist[0])]
                    try:
                        idx = barlist.index(barseq)
                        datalist[idx].append([seq])
                    except:
                        mismatcheddata.append([seq,'barcode not found'])
                if i%200000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismatcheddata,lowqualdata,i,new_folder,start)
                    mismatcheddata,lowqualdata = [],[]
                    datalist = datalist_empty
        if mode == 2:   # no quality control, conserved seq after barcode
            for line in f:
                i += 1
                if i%4 == 2:
                    seq = line.upper().strip(' \n')
                    try:
                        pos = seq.index(cons_aft)-len(barlist[0])
                    except:
                        mismatcheddata.append([seq,'conserved seq not found'])
                        continue
                    barseq = seq[pos:pos+len(barlist[0])]
                    try:
                        idx = barlist.index(barseq)
                        datalist[idx].append([seq])
                    except:
                        mismatcheddata.append([seq,'barcode not found'])
                if i%200000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismatcheddata,lowqualdata,i,new_folder,start)
                    mismatcheddata,lowqualdata = [],[]
                    datalist = datalist_empty
        if mode == 3:   # quality control, position search
            for line in f:
                i += 1
                if i%4 == 2:
                    seq = line.upper().strip(' \n')
                if i%4 == 0:
                    qual = line.strip('\n')
                    qualcheck = qualityfilter(qc_minq,qc_maxb,qual)
                    if qualcheck == False:
                        lowqualdata.append([seq])
                    else:
                        barseq = seq[pos:pos+len(barlist[0])]
                        try:
                            idx = barlist.index(barseq)
                            datalist[idx].append([seq])
                        except:
                            mismatcheddata.append([seq,'barcode not found'])
                if i%200000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismatcheddata,lowqualdata,i,new_folder,start)
                    mismatcheddata,lowqualdata = [],[]
                    datalist = datalist_empty
        if mode == 4:   # quality control, conserved seq before barcode
            for line in f:
                i += 1
                if i%4 == 2:
                    seq = line.upper().strip(' \n')
                if i%4 == 0:
                    qual = line.strip('\n')
                    qualcheck = qualityfilter(qc_minq,qc_maxb,qual)
                    if qualcheck == False:
                        lowqualdata.append([seq])
                    else:
                        try:
                            pos = seq.index(cons_bef)+len(cons_bef)
                        except:
                            mismatcheddata.append([seq,'conserved seq not found'])
                            continue
                        barseq = seq[pos:pos+len(barlist[0])]
                        try:
                            idx = barlist.index(barseq)
                            datalist[idx].append([seq])
                        except:
                            mismatcheddata.append([seq,'barcode not found'])
                if i%200000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismatcheddata,lowqualdata,i,new_folder,start)
                    mismatcheddata,lowqualdata = [],[]
                    datalist = datalist_empty
        if mode == 5:   # quality control, conserved seq after barcode
            for line in f:
                i += 1
                if i%4 == 2:
                    seq = line.upper().strip(' \n')
                if i%4 == 0:
                    qual = line.strip('\n')
                    qualcheck = qualityfilter(qc_minq,qc_maxb,qual)
                    if qualcheck == False:
                        lowqualdata.append([seq])
                    else:
                        try:
                            pos = seq.index(cons_aft)-len(barlist[0])
                        except:
                            mismatcheddata.append([seq,'conserved seq not found'])
                            continue
                        barseq = seq[pos:pos+len(barlist[0])]
                        try:
                            idx = barlist.index(barseq)
                            datalist[idx].append([seq])
                        except:
                            mismatcheddata.append([seq,'barcode not found'])
                if i%200000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismatcheddata,lowqualdata,i,new_folder,start)
                    mismatcheddata,lowqualdata = [],[]
                    datalist = datalist_empty
    csvfile = open(new_folder+'mismatched data.txt','a',newline='')
    write = csv.writer(csvfile)
    write.writerows(mismatcheddata)
    if lowqualdata != []:
        csvfile = open(path.replace('.fastq','/')+'low quality data.txt','a',newline='')
        write = csv.writer(csvfile)
        write.writerows(lowqualdata)
    for idx in range(len(barlist)):
        csvfile = open(new_folder+'Barcode_'+barlist[idx]+'.txt','a',newline='')
        write = csv.writer(csvfile)
        write.writerows(datalist[idx])
    csvfile.close()
    f.close()
    mismatcheddata,lowqualdata = [],[]
    datalist = datalist_empty
    result_label.config(text="Sorting finished.")
    print('>> Sorting finished.')
    print('>> Processed sequence count: '+str(int(i/4)))
    end = time.process_time()
    print('>> Processing time: '+str(end-start)+' s')
#################################################################################################################
#################################################################################################################
def submit2():
    print('>> Ambiguous search can recognise sequence with slight mutations (1 bp) on its barcode or conserved sequences around barcode.')
    file_path = file_path_entry.get()  # input file path
    barlist = barcode1_entry.get().upper().strip(' ,()\n').split(',') # barcode list
    qc_switch = qc1.get()  # quality check switch, on or off
    qc_minq = qc_entry.get() # minimal quality
    qc_maxb = qc2_entry.get() # maximal low quality bases
    lc_mth = dropdown.get() # barcode locating method
    pos = fp_entry.get() # barcode position (fixed)
    try:
        pos = int(pos)-1
    except:
        True
    cons_bef = cb_entry.get().upper()  # conserved seq before barcode 
    cons_aft = ca_entry.get().upper()  # conserved seq after barcode 
    mode = m_choice(qc_switch,lc_mth)
    folder_path = file_path.replace(file_path.split(os.sep)[-1],'')
    new_folder = file_path[:file_path.index('.f')]+os.sep
    barlength = len(barlist[0])
    datalist = []
    for i in range(len(barlist)):
        datalist.append([])
    dataliste = datalist
    mmd_path = new_folder+'mismatched data.txt'
    mismdata = []
    start = time.process_time()
    input_path = mmd_path.replace('.txt','_tmp.txt')
    os.renames(mmd_path,input_path)
    mcount,tcount = 0,0
    with open(input_path,'r',buffering = 1) as f:
        if mode%3 == 0:
            for line in f:
                mcount += 1
                seq = line.split(',')[0].strip(' ,\n')
                line = [line.split(',')[0].strip(' ,\n'),line.split(',')[1].strip(' ,\n')]
                bc = seq[pos:pos+barlength]
                bcp = seq[pos+1:pos+barlength+1]
                bcm = seq[pos-1:pos+barlength-1]
                for i in range(len(barlist)):
                    distance = min(LV.distance(bc,barlist[i]),LV.distance(bcp,barlist[i]),LV.distance(bcm,barlist[i]))
                    if distance <= 1:
                        datalist[i].append([seq])
                        tcount += 1
                    else:
                        mismdata.append(line)
                if i%100000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismdata,[],i,new_folder,start)
                    mismadata = []
                    datalist = dataliste
        elif mode%3 ==1:
            for line in f:
                mcount += 1
                seq = line.split(',')[0].strip(' ,\n')
                line = [line.split(',')[0].strip(' ,\n'),line.split(',')[1].strip(' ,\n')]
                pos = BClocate(cons_bef,seq)
                if pos == -1:
                    mismdata.append(line)
                    continue
                pos = pos + len(cons_bef)
                bc = seq[pos:pos+barlength]
                bcp = seq[pos+1:pos+barlength+1]
                bcm = seq[pos-1:pos+barlength-1]
                for i in range(len(barlist)):
                    distance = min(LV.distance(bc,barlist[i]),LV.distance(bcp,barlist[i]),LV.distance(bcm,barlist[i]))
                    if distance <= 1:
                        datalist[i].append([seq])
                        tcount += 1
                    else:
                        mismdata.append(line)
                if i%100000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismdata,[],i,new_folder,start)
                    mismadata = []
                    datalist = dataliste
        else:
            for line in f:
                mcount += 1
                seq = line.split(',')[0].strip(' ,\n')
                line = [line.split(',')[0].strip(' ,\n'),line.split(',')[1].strip(' ,\n')]
                pos = BClocate(cons_aft,seq)
                if pos == -1:
                    mismdata.append(line)
                    continue
                pos = pos - barlength
                bc = seq[pos:pos+barlength]
                bcp = seq[pos+1:pos+barlength+1]
                bcm = seq[pos-1:pos+barlength-1]
                for i in range(len(barlist)):
                    distance = min(LV.distance(bc,barlist[i]),LV.distance(bcp,barlist[i]),LV.distance(bcm,barlist[i]))
                    if distance <= 1:
                        datalist[i].append([seq])
                        tcount += 1
                    else:
                        mismdata.append(line)
                if i%100000 == 1 and i >= 2:
                    temp_output(datalist,barlist,mismdata,[],i,new_folder,start)
                    mismadata = []
                    datalist = dataliste   
    csvfile = open(mmd_path,'a',newline='')
    write = csv.writer(csvfile)
    write.writerows(mismdata)
    for idx in range(len(barlist)):
        csvfile = open(new_folder+'Barcode_'+barlist[idx]+'.txt','a',newline='')
        write = csv.writer(csvfile)
        write.writerows(datalist[idx])
    csvfile.close()
    f.close()
    mismdata = []
    datalist = dataliste
    os.remove(input_path)
    print('>> Ambiguous search finished.')
    print('>> Processed sequence count: '+str(mcount))
    print('>> Recognised and transferred '+str(tcount)+' sequences from mismatched data to barcode files.')
    end = time.process_time()
    print('>> Processing time: '+str(end-start)+' s')
    result2_label.config(text="Ambiguous search finished.")
def turnoffentry1(x):
    a = qc1.get()
    if a == 'Skip Quality Control':
        qc_entry.config(state = 'disabled')
        qc2_entry.config(state = 'disabled')
    else:
        qc_entry.config(state = 'normal')
        qc2_entry.config(state = 'normal')
def turnoffentry2(x):
    a = dropdown.get()
    if a == 'Start from a Fixed Position':
        cb_entry.config(state = 'disabled')
        ca_entry.config(state = 'disabled')
        fp_entry.config(state = 'normal')
    elif a == 'Find with Conserved Seq before':
        ca_entry.config(state = 'disabled')
        fp_entry.config(state = 'disabled')
        cb_entry.config(state = 'normal')
    else:
        cb_entry.config(state = 'disabled')
        fp_entry.config(state = 'disabled')
        ca_entry.config(state = 'normal')
def show_default_BC():
    barcode1_entry.delete(0, tk.END)
    barcode1_entry.insert(0,'GCTCAGGA,AGGAGTCC,CATGCCTA,GTAGAGAG,TCATGAGC,CCTGAGAT,TAGCGAGT,TACTACGC,GCAGCGTA,ACTGATCG,TAGCTGCA,GACGTCGA')
#========================= define GUI functions ==============================

#====================== GUI interface ====================================
# Create the main window
root = tk.Tk()
root.title("NGS peptide analysis - S1 Barcode Sorting Tool")

# Create entry field and button for input data file selection dialogue
file_path_entry = tk.Entry(root, width = 85)
file_path_entry.grid(row=0, column=0, columnspan=6, padx=10, pady=5)
select_file_button = tk.Button(root, text="    Select File and Preview  ", command=preview_fastq)
select_file_button.grid(row=0, column=6, columnspan = 2, padx=10, pady=5)

# Create a text box to display the first 5 lines of the text file
text_box = tk.Text(root, height=6, width=120)
text_box.grid(row=3, column=0, columnspan=8, padx=10, pady=5)

# Create labels and entry fields for Barcode
barcode1_label = tk.Label(root, text="Barcodes (split with comma), e.g., GCTCAGGA,AGGAGTCC,CATGCCTA ")
barcode1_label.grid(row=4, column=0, columnspan=6, padx=10, pady=5)
defaultbc = tk.Button(root,text = '  Use Default Barcode Set  ', command = show_default_BC)
defaultbc.grid(row=4, column=6, columnspan=2, padx=10, pady=5)
barcode1_entry = tk.Entry(root, width= 120)
barcode1_entry.grid(row=5, column=0, columnspan=8, padx=10, pady=5)

# dropbox for QC
qc1 = ttk.Combobox(root, values=['Skip Quality Control','Perform Quality Control'],width = 25)
qc1.current(0) 
qc1.bind("<<ComboboxSelected>>", turnoffentry1)
qc1.grid(row = 6, column = 0, columnspan = 2, padx = 25)
qc_label = tk.Label(root, text=' Minimal Quality Threshold:                     ' )
qc_label.grid(row=6, column=3, columnspan=2, padx=5, pady=5)
qc_entry = tk.Entry(root, width= 5)
qc_entry.insert(0, "18")
qc_entry.grid(row=6, column=4, columnspan=1, padx=5, pady=5)
qc2_label = tk.Label(root, text= 'Maximal Low Quality Bases:' )
qc2_label.grid(row=6, column=6, columnspan=1, padx=5, pady=5)
qc2_entry = tk.Entry(root, width= 5)
qc2_entry.insert(0, "3")
qc2_entry.grid(row=6, column=7, columnspan=1, padx=15, pady=10)
qc_entry.config(state = 'disabled')
qc2_entry.config(state = 'disabled')

hyfen = tk.Label(root,text = '* ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ')
hyfen.grid(row=7,column = 0, columnspan = 8, padx = 10, pady = 5)

# Create a text box to display default seq format
tpl_box = tk.Text(root, height=4, width=80)
tpl_box.insert(tk.END,'   Barcode        N-ter conserved              Library       C-ter conserved\n  ======== ---------------------------- ==================== ---------------\n  AGGAGTCC ATTCTATGCGGCCCAGCCGGCCATGGCG NNKTGC (NNK)n TGTNNK GGTTCTGGCGCTGAA\n            |F||Y||A||A||Q||P||A||M||A| |X||C|  |X|n  |C||X| |G||S||G||A||E|')
tpl_box.grid(row = 8, column = 2, columnspan = 6)
tpl_label = tk.Label(root,text = 'The default format\n of a sequence:')
tpl_label.grid(row = 8, column =0, rowspan = 3)

hyfen2 = tk.Label(root,text = '* ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ')
hyfen2.grid(row=11,column = 0, columnspan = 8, padx = 10, pady = 5)

# Create dropboxes for selection of barcode finding mode 
droplabel = tk.Label(root, text="Mode of Locating Barcode:")
droplabel.grid(row=12, column=0, columnspan=3, padx=10, pady=5)
dropdown = ttk.Combobox(root, values=['Start from a Fixed Position','Find with Conserved Seq before','Find with Conserved Seq after'],width = 25)
dropdown.current(0) 
dropdown.bind("<<ComboboxSelected>>", turnoffentry2)
dropdown.grid(row=13, column=0, columnspan=3, padx=10, pady=5)

fp_label = tk.Label(root, text='Start from Position:' )
fp_label.grid(row=12, column=3, columnspan=1, padx=5, pady=5)
fp_entry = tk.Entry(root, width= 5)
fp_entry.insert(0, "1")
fp_entry.grid(row=13, column=3, columnspan=1, padx=5, pady=5)

cb_label = tk.Label(root, text='/   Conserved Seq before Barcode:' )
cb_label.grid(row=12, column=4, columnspan=1, padx=5, pady=5)
cb_entry = tk.Entry(root)
cb_entry.grid(row=13, column=4, columnspan=2, padx=5, pady=5)

ca_label = tk.Label(root, text='/   Conserved Seq after Barcode:' )
ca_label.grid(row=12, column=6, columnspan=2, padx=5, pady=5)
ca_entry = tk.Entry(root)
ca_entry.grid(row=13, column=6, columnspan=2, padx=5, pady=5)
cb_entry.config(state = 'disabled')
ca_entry.config(state = 'disabled')

# Create a button to submit the input
submit_button = tk.Button(root, text=">>>>> Step 1: Submit and Sort <<<<<", width = 50, command=submit)
submit_button.grid(row=14, column=0, columnspan=4, padx=10, pady=10)
# Create a button to submit ambiguous search
submit2_button = tk.Button(root, text=">>>> Step 2: Ambiguous Search (Optional) <<<<", width = 50, command=submit2)
submit2_button.grid(row=15, column=0, columnspan=4, padx=10, pady=10)
# Create a label to display computing status
result_label = tk.Label(root, text="")
result_label.grid(row=14, column=4, columnspan=3, pady=10)
result2_label = tk.Label(root, text="")
result2_label.grid(row=15, column=4, columnspan=3, pady=10)
root.mainloop()
#====================== GUI interface ====================================